# Casino Floor Mix Optimization

## Import
    Import delle librerie utilizzate all'interno del notebook e del dataset di riferimento per il progetto.
    Vengono create 3 funzioni per il calcolo della profitability di ogni slot machine, quantità che verrà utilizzata
    successivamente per calcolare la disposizione ottimale delle stesse.
    

In [265]:
import os
from datetime import datetime
import datetime
import pandas as pd
import numpy as np
import pulp
from pulp import *

def Daily_avg_revenue(df):
    tmp = []
    for row in df.iterrows():
        tmp.append((row[1].GrossRevenuePerMachine /30))
    return tmp

def Daily_avg_plays(df):
    tmp = []
    for row in df.iterrows():
        tmp.append((row[1].PlaysPerMachine /30))
    return tmp

def Daily_profitability(df):
    tmp = []
    for row in df.iterrows():
        tmp.append(row[1].Daily_avg_rev * row[1].Daily_avg_plays)
    return tmp

folder_path = os.getcwd()
file_ = "\\Lucky Duck Entertainment revenue 2013.xls"

x1 = pd.ExcelFile(folder_path + file_)
df1 = x1.parse(u'#LN00072')

section = ["Boundary","Entrance", "Interior", "RestaurantPlaza"]
date = ['2011-10-01','2011-11-01','2011-12-01','2012-01-01',
       '2012-02-01','2012-03-01','2012-04-01','2012-05-01','2012-06-01',
       '2012-07-01','2012-08-01']

df1['Month'] = pd.to_datetime(df1['Month'],format='%d.%m.%Y') 
df1['Daily_avg_rev'] = Daily_avg_revenue(df1)
df1['Daily_avg_plays'] = Daily_avg_plays(df1)
df1['Daily_profitability'] = Daily_profitability(df1)
df1['Daily_avg_rev'] = df1.Daily_avg_rev.apply(lambda x: '%.2f' % x)
df1['Daily_avg_plays'] = df1.Daily_avg_plays.apply(lambda x: '%.2f' % x)

In [266]:
df1.head()

,MachineName,Casino,Section,Denomination,MachineType,Manufacturer,Model,Month,GrossRevenue,FcstLower,FcstUpper,NoMachines,Plays,GrossRevenuePerMachine,PlaysPerMachine,Diff_w_Upper,Diff_w_Lower,Daily_avg_rev,Daily_avg_plays,Daily_profitability
0,Breakstone,Aries,RestaurantPlaza,0.05,video,Realm,Aeolian PRO Vision,2012-05-01,95929,75124,117941,1,280507,95929.00,280507.00,12267,12208,3197.63,9350.23,2.989862e+07
1,Bhaga Steinbok,Aries,Boundary,0.01,reel,Tsunami,Pedalfer,2011-11-01,13204,3412,22951,2,710245,6602.00,355122.50,5171,5183,220.07,11837.42,2.605021e+06
2,Lepiotaceae,Aries,RestaurantPlaza,1.00,video,Tsunami,Lantana,2011-10-01,337417,287271,534993,4,75143,84354.25,18785.75,101055,101116,2811.81,626.19,1.760731e+06
3,Isopectic Hypethral,Libra,Interior,2.00,video,Entellus,Mirier,2012-04-01,83436,47285,113868,1,29433,83436.00,29433.00,47728,47780,2781.20,981.10,2.728635e+06
4,Isopectic Hypethral,Libra,Interior,0.25,video,Entellus,Mirier,2012-01-01,80974,46798,88446,1,29281,80974.00,29281.00,43557,43600,2699.13,976.03,2.634444e+06


## Casino Aries
    In questa sezione viene trattato il problema di ottimizzazione del mix di slot machine per quanto concerne 
    il Casino Aries
    

In [267]:
df_aries = df1[df1.Casino == 'Aries']

### Controllo slot machine
    Controllo del dataset per essere sicuri che nelle date successive al 2011-09-01 non siano introdotte nuove slot
    machine all'interno del casino

In [268]:
v1 = df_aries[df_aries.Month == '2011-09-01'].MachineName.values

new_machine = []
for d in date:
    v_i = df_aries[df_aries.Month == d].MachineName.values
    for v in v_i:
        if (v in v1) == False:
            new_machine.append(v)
if len(new_machine) == 0:
    print "No new slot machines"
else:
    print "Some new slot machines"
    

No new slot machines


### Capienza section e Casino
    Appurato che nelle osservazioni del 2011-09-01 sono presenti tutte le slot machine del casino si può estrarre
    la capienza del Casinò e delle singole section.

In [269]:
Nslot_aries = df_aries[df_aries.Month == '2011-09-01'].NoMachines.sum()
Nslot_aries_boundary = df_aries[(df_aries.Month == '2011-09-01') & (df_aries.Section == section[0])].NoMachines.sum()
Nslot_aries_entrance = df_aries[(df_aries.Month == '2011-09-01') & (df_aries.Section == section[1])].NoMachines.sum()
Nslot_aries_interior = df_aries[(df_aries.Month == '2011-09-01') & (df_aries.Section == section[2])].NoMachines.sum()
Nslot_aries_restaurant = df_aries[(df_aries.Month == '2011-09-01') & (df_aries.Section == section[3])].NoMachines.sum()

print "Capienza Sala {0}: {1}".format(section[0], Nslot_aries_boundary)
print "Capienza Sala {0}: {1}".format(section[1], Nslot_aries_entrance)
print "Capienza Sala {0}: {1}".format(section[2], Nslot_aries_interior)
print "Capienza Sala {0}: {1}".format(section[3], Nslot_aries_restaurant)
print "Capienza totale Casinò Aries: {0}".format(Nslot_aries)

Capienza Sala Boundary: 195
Capienza Sala Entrance: 219
Capienza Sala Interior: 248
Capienza Sala RestaurantPlaza: 187
Capienza totale Casinò Aries: 849


### Inventario slot Machine 

    All'interno del dataset Aries_wh sono presenti tutte le differenti slot machines che sono presenti nel Casinò

In [270]:
#group by per sommare le righe duplicate
# le righe sono duplicate se corrispondono MachineName, Section, Denomination, MachineType, Model, Manufacturer e Month
df_aries = df_aries.groupby(by = ['MachineName','Section','Denomination','MachineType','Manufacturer','Model','Month'], as_index = False).sum()

Aries_wh = df_aries[df_aries.Month == '2011-09-01']
Aries_wh = Aries_wh.groupby(by = ['MachineName','Manufacturer','Model','MachineType','Denomination'], as_index = False)['NoMachines'].sum()

Aries_wh.reset_index(inplace = True)
Aries_wh = Aries_wh.loc[:,'MachineName':'NoMachines']
Aries_wh.head()

,MachineName,Manufacturer,Model,MachineType,Denomination,NoMachines
0,Abidjan,Tsunami,Hotspur,video,0.01,1
1,Achaian,Tsunami,Predate,video,0.01,1
2,Achromatophil,Global,Filum,video,2.00,1
3,Aegiceras,Global,Vesta,video,2.00,5
4,Africander,Global,Filum,video,2.00,1


### Profitti slot machine Aries

    Nel DataFrame final_prof_aries troviamo i dati delle profitability calcolate per ogni singola configurazione di slot
    machine.

In [271]:
#Estraggo dal dataset iniziale solo le colonne che mi servono
#Escludo tutti i dati che non si riferiscono alla singola slot
prof_aries = df_aries.iloc[:,[0,1,2,3,4,5,6,12,13,15]]

#Aggiungo una colonna settata a 1 che dopo aver fatto la groupby per le
#diverse configurazioni conterrà la somma delle righe che hanno la stessa
#configurazione
prof_aries.loc[:,'count'] = 1
df_prof_aries = prof_aries.groupby(by = ['MachineName','Section','Denomination','Manufacturer','Model'], as_index = False).sum()

#lambda function per dividere poi le colonne che si sono sommate in modo da mantenere
#sempre i dati riferiti alla singola slot
divide_count = lambda x: x/df_prof_aries['count']

#Applico la funzione alle colonne che mi serve dividere
df_prof_aries.loc[:,'GrossRevenuePerMachine':'count'] = df_prof_aries.loc[:,'GrossRevenuePerMachine':'count'].apply(divide_count)

#Arrotondo i profitti alla seconda cifra decimale
df_prof_aries.loc[:,'GrossRevenuePerMachine':'Daily_profitability'] = df_prof_aries.loc[:,'GrossRevenuePerMachine':'Daily_profitability'].applymap(lambda x: '%.2f' % x)

#Elimino la colonna count creata in precedenza
final_prof_aries = df_prof_aries.loc[:,'MachineName':'Daily_profitability']
final_prof_aries.head()

,MachineName,Section,Denomination,Manufacturer,Model,GrossRevenuePerMachine,PlaysPerMachine,Daily_profitability
0,Abidjan,Boundary,0.01,Tsunami,Hotspur,3674.00,85687.00,349793.38
1,Abidjan,Entrance,0.01,Tsunami,Hotspur,6278.50,70597.50,528933.29
2,Abidjan,Interior,0.01,Tsunami,Hotspur,9909.33,58934.67,658920.16
3,Achaian,Boundary,0.01,Tsunami,Predate,2407.67,272114.33,666044.51
4,Achaian,RestaurantPlaza,0.01,Tsunami,Predate,23868.67,316710.00,9516418.53


# Modello
## Preparazione dei dati

### Variabili di decisione


Creazione del vettore per le decision variables, una per ogni slot machine presente nel casinò, in totale 849.

Per comodità di lettura ogni decision variable sarà scritta nella forma $x_i$ con $i \in [1..849]$ dal momento che ogni singola slot è univocamente identificata dalla tripla $\{ MachineName, Model, Denomination \}$ e renderebbe difficile la lettura usare questa notazione.

Verrà creata una tabella apposita per conservare la corrispondeza tra decision variable e singola slot machine.

In [272]:
decision_variables = []

for i in range(Aries_wh.NoMachines.sum()):
    decision_variables.append("x_" + str(i+1))

slot_machines = []
for rownum,row in Aries_wh.iterrows():
    if row.NoMachines != 1:
        for i in range(row.NoMachines):
            tmp = [row.MachineName, row.Model, row.Denomination]
            slot_machines.append(tmp)
    else:
        tmp = [row.MachineName, row.Model, row.Denomination]
        slot_machines.append(tmp)

In [273]:
c_names = ["DecisionVar","MachineName","Model","Denomination"]
dec_var = np.insert(slot_machines,0,decision_variables, axis = 1)
dec_var_df = pd.DataFrame(dec_var, columns = c_names)
dec_var_df.head()

,DecisionVar,MachineName,Model,Denomination
0,x_1,Abidjan,Hotspur,0.01
1,x_2,Achaian,Predate,0.01
2,x_3,Achromatophil,Filum,2.0
3,x_4,Aegiceras,Vesta,2.0
4,x_5,Aegiceras,Vesta,2.0


### Matrice dei coefficienti di profitto

Per poter utilizzare i dati dei profitti medi giornalieri si crea una matrice formata da N righe quanto il totale delle slot machine presenti e da M colonne quante le section del casinò

In [274]:
# La seguente funzione prendere in ingresso MachineName, Denomination, Model e Section di una slot machine e ne restituisce
# il valore della profitability se esiste, 0 altrimenti
def prof_value(name, den, model, section):
    riga = final_prof_aries[(final_prof_aries.MachineName == name) & (final_prof_aries.Model == model) & (final_prof_aries.Denomination == den)]
    if section in riga.Section.values:
        value = riga[riga.Section == section].Daily_profitability.values
        return float(value[0])
    else:
        return 1

In [275]:
#Creo un matrice a cui andrò ad aggiungere riga per riga i valori dei profitti
prof_matrix = np.array((0,0,0,0))
for rownum,row in Aries_wh.iterrows():
    name = row.MachineName
    den = row.Denomination
    model = row.Model
    #Se ho più di 1 slot per configurazione creo tante righe quante le slot
    if row.NoMachines != 1:
        p_bound = prof_value(name, den, model, 'Boundary')
        p_entrance = prof_value(name, den, model, 'Entrance')
        p_interior = prof_value(name, den, model, 'Interior')
        p_rest = prof_value(name, den, model, 'RestaurantPlaza')
        for i in range(row.NoMachines):
            tmp = np.array([p_bound,p_entrance,p_interior,p_rest])
            prof_matrix = np.vstack((prof_matrix, tmp))
    else:
        p_bound = prof_value(name, den, model, 'Boundary')
        p_entrance = prof_value(name, den, model, 'Entrance')
        p_interior = prof_value(name, den, model, 'Interior')
        p_rest = prof_value(name, den, model, 'RestaurantPlaza')
        tmp = np.array([p_bound,p_entrance,p_interior,p_rest])
        prof_matrix = np.vstack((prof_matrix, tmp))
        
prof_matrix = np.delete(prof_matrix, 0, 0)

In [276]:
#Trasformo la matrice in un dataset in modo da poterci accedere con 
#i valori delle section e decision variables
profit_coeff = pd.DataFrame(prof_matrix, columns = section, index = decision_variables)

## Creazione modello e inserimento funzione obiettivo

$
   X_{j,i} = 
    \begin{cases}
        1 & \quad \text{se la slot} ~ i ~\text{è presente nella sezione}~j\\
        0 & \quad \text{altrimenti}
    \end{cases}
\
$

$
    P_{j,i} = profitability[j,i] ~\text{dove profitability è la matrice dei profitti creata precedentemente}
$

In [277]:
prob = pulp.LpProblem('Floor Mix Optimization at Aries Casino', pulp.LpMaximize)

#La funzione LpVariable.dicts crea un dizionario con tutte le possibili combinazioni tra i e j
is_machine_var = pulp.LpVariable.dicts("is there",
                                      [(j,i) for j in section 
                                             for i in decision_variables],
                                      0,1, LpBinary)

Funzione obiettivo:
   
$ \displaystyle \sum_{j \in section} \sum_{i \in decision_variables} X_{j,i}P_{j,i}$

In [278]:
prob += (
    pulp.lpSum(
        [profit_coeff[j][i] * is_machine_var[(j,i)]]
        for j in section for i in decision_variables
    )
)

## Vincoli del modello

Vengono aggiunti al modello i vincoli di capienza delle singole stanze e un vincolo per ogni variabile decisionale poichè rappresentando la singola slot presente nel Casinò può essere presente solamente in una delle 4 sezioni.

In [279]:
prob += pulp.lpSum([is_machine_var['Boundary', i] for i in decision_variables]) == 195 #Boundary
prob += pulp.lpSum([is_machine_var['Entrance', i] for i in decision_variables]) == 219 #Entrance
prob += pulp.lpSum([is_machine_var['Interior', i] for i in decision_variables]) == 248 #Interior
prob += pulp.lpSum([is_machine_var['RestaurantPlaza', i] for i in decision_variables]) == 187 #RestaurantPlaza
#La somma dei valori di Xji nelle 4 sezioni deve essere pari a 1
for i in decision_variables:
    prob += is_machine_var['Boundary',i] + is_machine_var['Entrance',i] + is_machine_var['Interior',i] + is_machine_var['RestaurantPlaza',i] == 1

## Calcolo Modello e soluzione ottimale

In [280]:
prob.solve()

1

Controllo che la capienza delle singole sezioni sia corretta dopo la risoluzione del modello

In [281]:
tot = 0
for var in is_machine_var:
    if var[0] == 'Boundary':
        var_value = is_machine_var[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in Boundary: {0}".format(tot)

Numero di Slot in Boundary: 195.0


In [282]:
tot = 0
for var in is_machine_var:
    if var[0] == 'Entrance':
        var_value = is_machine_var[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in Entrance: {0}".format(tot)

Numero di Slot in Entrance: 219.0


In [283]:
tot = 0
for var in is_machine_var:
    if var[0] == 'Interior':
        var_value = is_machine_var[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in Interior: {0}".format(tot)

Numero di Slot in Interior: 248.0


In [284]:
tot = 0
for var in is_machine_var:
    if var[0] == 'RestaurantPlaza':
        var_value = is_machine_var[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in RestaurantPlaza: {0}".format(tot)

Numero di Slot in RestaurantPlaza: 187.0


In [285]:
opt_result = prob.solve()

assert opt_result == pulp.LpStatusOptimal
print("Status: ", LpStatus[prob.status])
print("Optimal solution: ", round(prob.objective.value(),2))
# print("Individual variables: ")
# for v in prob.variables():
#     print(v.name, " = ", v.varValue)

('Status: ', 'Optimal')
('Optimal solution: ', 1896884720.64)


### Confronto risultati con Settembre 2011

Confronto tra la disposizione e la profitability del casino del 2011-09-01 con quella ottimale calcolata


In [187]:
table_col_names = ['MachineName','B_11', 'E_11', 'I_11', 'R_11','B_12', 'E_12', 'I_12', 'R_12']
df_aries_sept = df_aries[df_aries.Month == '2011-09-01']

old_disposizione = np.array((0,0,0,0))

for rownum,row in dec_var_df.iterrows():
    name = row.MachineName
    model = row.Model
    den = row.Denomination
    old_disp = df_aries_sept[(df_aries_sept.MachineName == name) & (df_aries_sept.Model == model) & (df_aries_sept.Denomination == float(den))].Section.values[0]
    if old_disp == 'Boundary':
        old_tmp = np.array((1,0,0,0))
    elif old_disp == 'Entrance':
        old_tmp = np.array((0,1,0,0))
    elif old_disp == 'Interior':
        old_tmp = np.array((0,0,1,0))
    else:
        old_tmp = np.array((0,0,0,1))
    old_disposizione = np.vstack((old_disposizione, old_tmp))
old_disposizione = np.delete(old_disposizione, 0, 0)
name_df = dec_var_df.MachineName
old_disposizione_df = pd.DataFrame(old_disposizione)

old_disposizione_df = pd.concat([old_disposizione_df, name_df], axis = 1)
old_disposizione_df['index'] = decision_variables
old_disposizione_df = old_disposizione_df.set_index('index')
cols = old_disposizione_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
old_disposizione_df = old_disposizione_df[cols]
old_disposizione_df[4] = 0
old_disposizione_df[5] = 0
old_disposizione_df[6] = 0
old_disposizione_df[7] = 0

old_disposizione_df.columns = table_col_names
old_disposizione_df['B_11'] = old_disposizione_df.B_11.astype(int)
old_disposizione_df['E_11'] = old_disposizione_df.E_11.astype(int)
old_disposizione_df['I_11'] = old_disposizione_df.I_11.astype(int)
old_disposizione_df['R_11'] = old_disposizione_df.R_11.astype(int)
old_disposizione_df['B_12'] = old_disposizione_df.B_12.astype(int)
old_disposizione_df['E_12'] = old_disposizione_df.E_12.astype(int)
old_disposizione_df['I_12'] = old_disposizione_df.I_12.astype(int)
old_disposizione_df['R_12'] = old_disposizione_df.R_12.astype(int)

for v in prob.variables():
    indice = str(v)[23:-2]
    if indice.startswith('aza'):
        indice = str(v)[30:-2]
    if str(v)[11:19] == 'Boundary':
        sect = 'B_12'
    elif str(v)[11:19] == 'Entrance':
        sect = 'E_12'
    elif str(v)[11:19] == 'Interior':
        sect = 'I_12'
    else:
        sect = 'R_12'
    old_disposizione_df.at[indice, sect] = int(v.varValue)

Rappresentazione sotto forma di DataFrame della disposizione iniziale e ottimale delle slot machines

In [188]:
colored_table = old_disposizione_df.loc[:,'B_11':'R_12'].style.applymap(lambda x: 'background-color: green' if x == 1 else 'background-color: lightgray')
colored_table

Le seguenti righe di codice servono per aggiungere al DataFrame iniziale df_aries le righe corrispondenti ai dati della disposizione ottimale ed esportare il nuovo dataset in formato csv

In [302]:
#Aries_wh ci attacco la section dal modello e poi prendo la profitability calcolata
dec_var_df_ = dec_var_df[:]
dec_var_df_ = dec_var_df_.set_index('DecisionVar')
dec_var_df_['Section'] = ''
#dec_var_df_['Daily_profitability'] = 0.0

def change_b(x):
    if x == 1:
        return 'Boundary'
    
def change_e(x):
    if x == 1:
        return 'Entrance'

def change_i(x):
    if x == 1:
        return 'Interior'
    
def change_r(x):
    if x == 1:
        return 'RestaurantPlaza'
    
old_disposizione_df_ = old_disposizione_df[:]
old_disposizione_df_['B_12'] = old_disposizione_df_['B_12'].apply(change_b)
old_disposizione_df_['E_12'] = old_disposizione_df_['E_12'].apply(change_e)
old_disposizione_df_['I_12'] = old_disposizione_df_['I_12'].apply(change_i)
old_disposizione_df_['R_12'] = old_disposizione_df_['R_12'].apply(change_r)

for rownum,row in dec_var_df_.iterrows():
    row['Section'] = old_disposizione_df_.loc[rownum,'B_12':'R_12'].values[ old_disposizione_df_.loc[rownum,'B_12':'R_12'].values != np.array(None)][0]

dec_var_df_['Daily_profitability'] = 0.0
for rownum,row in dec_var_df_.iterrows():
    name = row.MachineName
    model = row.Model
    deno = row.Denomination
    sec = row.Section
    try:
        value = final_prof_aries[(final_prof_aries.MachineName == name) & (final_prof_aries.Model == model) & (final_prof_aries.Denomination == float(deno)) & (final_prof_aries.Section == sec)].Daily_profitability.values[0]  
    except IndexError:
        value = final_prof_aries[(final_prof_aries.MachineName == name) & (final_prof_aries.Model == model) & (final_prof_aries.Denomination == float(deno))].Daily_profitability.values[0]
    dec_var_df_.at[rownum,'Daily_profitability'] = value
dec_var_df_['NoMachines'] = 1

,MachineName,Model,Denomination,Section,Daily_profitability,NoMachines
DecisionVar,,,,,,
x_1,Abidjan,Hotspur,0.01,Entrance,528933.29,1
x_2,Achaian,Predate,0.01,RestaurantPlaza,9516418.53,1
x_3,Achromatophil,Filum,2.0,RestaurantPlaza,3095905.26,1
x_4,Aegiceras,Vesta,2.0,Interior,3152466.50,1
x_5,Aegiceras,Vesta,2.0,Interior,3152466.50,1
x_6,Aegiceras,Vesta,2.0,Interior,3152466.50,1
x_7,Aegiceras,Vesta,2.0,Interior,3152466.50,1
x_8,Aegiceras,Vesta,2.0,Interior,3152466.50,1
x_9,Africander,Filum,2.0,RestaurantPlaza,4119620.72,1


In [305]:
dec_var_df_final = dec_var_df_.groupby(by = ['MachineName','Model','Denomination','Section'], as_index = False)['Daily_profitability','NoMachines'].sum()
dec_var_df_final['Month'] = '2012-09-01'
dec_var_df_final['Casino'] = 'Aries'

dec_var_df_final = dec_var_df_final[['MachineName','Section','Denomination','Model','Month','NoMachines','Daily_profitability']]
dec_var_df_final['Daily_profitability'] = dec_var_df_final.Daily_profitability / dec_var_df_final.NoMachines

df_aries[['MachineName','Section','Denomination','Model','Month','NoMachines','Daily_profitability']]
df_ariescopia = df_aries[['MachineName','Section','Denomination','Model','Month','NoMachines','Daily_profitability']]

TUTTO_ARIES = pd.concat([df_ariescopia,dec_var_df_final])
TUTTO_ARIES['Profit'] = TUTTO_ARIES.NoMachines*TUTTO_ARIES.Daily_profitability

In [306]:
file_name = folder_path + '\\Aries_casino.csv'     
TUTTO_ARIES.to_csv(file_name, encoding = 'utf-8', index = False)

## Casino Libra
    In questa sezione viene trattato il problema di ottimizzazione del mix di slot machine per quanto concerne 
    il Casino Libra
    

In [309]:
df_libra = df1[df1.Casino == 'Libra']

### Controllo slot machine
    Controllo del dataset per essere sicuri che nelle date successive al 2011-09-01 non siano introdotte nuove slot
    machine all'interno del casino

In [310]:
v1 = df_libra[df_libra.Month == '2011-09-01'].MachineName.values

new_machine = []
for d in date:
    v_i = df_libra[df_libra.Month == d].MachineName.values
    for v in v_i:
        if (v in v1) == False:
            #print(' '.join([v,d]))
            new_machine.append(v)
if len(new_machine) == 0:
    print "No new slot machines"
else:
    print "Some new slot machines"
    

No new slot machines


### Capienza section e Casino
    Appurato che nelle osservazioni del 2011-09-01 sono presenti tutte le slot machine del casino si può estrarre
    la capienza del Casinò e delle singole section.

In [311]:
Nslot_libra = df_libra[df_libra.Month == '2011-09-01'].NoMachines.sum()
Nslot_libra_boundary = df_libra[(df_libra.Month == '2011-09-01') & (df_libra.Section == section[0])].NoMachines.sum()
Nslot_libra_entrance = df_libra[(df_libra.Month == '2011-09-01') & (df_libra.Section == section[1])].NoMachines.sum()
Nslot_libra_interior = df_libra[(df_libra.Month == '2011-09-01') & (df_libra.Section == section[2])].NoMachines.sum()
Nslot_libra_restaurant = df_libra[(df_libra.Month == '2011-09-01') & (df_libra.Section == section[3])].NoMachines.sum()

print "Capienza Sala {0}: {1}".format(section[0], Nslot_libra_boundary)
print "Capienza Sala {0}: {1}".format(section[1], Nslot_libra_entrance)
print "Capienza Sala {0}: {1}".format(section[2], Nslot_libra_interior)
print "Capienza Sala {0}: {1}".format(section[3], Nslot_libra_restaurant)
print "Capienza totale Casinò Aries: {0}".format(Nslot_libra)

Capienza Sala Boundary: 53
Capienza Sala Entrance: 56
Capienza Sala Interior: 39
Capienza Sala RestaurantPlaza: 82
Capienza totale Casinò Aries: 230


### Inventario slot Machine 

    All'interno del dataset Aries_wh sono presenti tutte le differenti slot machines che sono presenti nel Casinò

In [312]:
#group by per sommare le righe duplicate
# le righe sono duplicate se corrispondono MachineName, Section, Denomination, MachineType, Model, Manufacturer e Month
df_libra = df_libra.groupby(by = ['MachineName','Section','Denomination','MachineType','Manufacturer','Model','Month'], as_index = False).sum()

Libra_wh = df_libra[df_libra.Month == '2011-09-01']
Libra_wh = Libra_wh.groupby(by = ['MachineName','Manufacturer','Model','MachineType','Denomination'], as_index = False)['NoMachines'].sum()

Libra_wh.reset_index(inplace = True)
Libra_wh = Libra_wh.loc[:,'MachineName':'NoMachines']
Libra_wh.head()


,MachineName,Manufacturer,Model,MachineType,Denomination,NoMachines
0,Achaian,Tsunami,Predate,video,0.01,1
1,Achromatophil,Global,Filum,video,2.00,1
2,Africander,Global,Filum,video,0.01,1
3,Aleppo,Realm,Aeolian PRO V2,video,2.00,1
4,Amelanchier,Global,Anosmia,video,0.02,1


### Profitti slot machine Libra

    Nel DataFrame final_prof_libra troviamo i dati delle profitability calcolate per ogni singola configurazione di slot
    machine.

In [313]:
#Estraggo dal dataset iniziale solo le colonne che mi servono
#Escludo tutti i dati che non si riferiscono alla singola slot
prof_libra = df_libra.iloc[:,[0,1,2,3,4,5,6,12,13,15]]

#Aggiungo una colonna settata a 1 che dopo aver fatto la groupby per le
#diverse configurazioni conterrà la somma delle righe che hanno la stessa
#configurazione
prof_libra.loc[:,'count'] = 1
df_prof_libra = prof_libra.groupby(by = ['MachineName','Section','Denomination','Manufacturer','Model'], as_index = False).sum()

#lambda function per dividere poi le colonne che si sono sommate in modo da mantenere
#sempre i dati riferiti alla singola slot
divide_count = lambda x: x/df_prof_libra['count']

#Applico la funzione alle colonne che mi serve dividere
df_prof_libra.loc[:,'GrossRevenuePerMachine':'count'] = df_prof_libra.loc[:,'GrossRevenuePerMachine':'count'].apply(divide_count)

#Arrotondo i profitti alla seconda cifra decimale
df_prof_libra.loc[:,'GrossRevenuePerMachine':'Daily_profitability'] = df_prof_libra.loc[:,'GrossRevenuePerMachine':'Daily_profitability'].applymap(lambda x: '%.2f' % x)

#Elimino la colonna count creata in precedenza
final_prof_libra = df_prof_libra.loc[:,'MachineName':'Daily_profitability']
final_prof_libra.head()

,MachineName,Section,Denomination,Manufacturer,Model,GrossRevenuePerMachine,PlaysPerMachine,Daily_profitability
0,Achaian,Entrance,0.01,Tsunami,Predate,2251.00,354956.00,749952.21
1,Achaian,Interior,0.01,Tsunami,Predate,21894.00,341681.33,8468034.59
2,Achaian,RestaurantPlaza,0.01,Tsunami,Predate,12443.00,270167.00,3735208.87
3,Achromatophil,Boundary,2.00,Global,Filum,6725.50,203342.25,1622461.89
4,Achromatophil,Interior,2.00,Global,Filum,20086.00,103070.67,2042486.90


# Modello
## Preparazione dei dati

### Variabili di decisione


Creazione del vettore per le decision variables, una per ogni slot machine presente nel casinò, in totale 230

Per comodità di lettura ogni decision variable sarà scritta nella forma $x_i$ con $i \in [1..230]$ dal momento che ogni singola slot è univocamente identificata dalla tripla $\{ MachineName, Model, Denomination \}$ e renderebbe difficile la lettura usare questa notazione.

Verrà creata una tabella apposita per conservare la corrispondeza tra decision variable e singola slot machine.

In [314]:
decision_variables_libra = []

for i in range(Libra_wh.NoMachines.sum()):
    decision_variables_libra.append("x_" + str(i+1))

slot_machines_libra = []
for rownum,row in Libra_wh.iterrows():
    if row.NoMachines != 1:
        for i in range(row.NoMachines):
            tmp = [row.MachineName, row.Model, row.Denomination]
            slot_machines_libra.append(tmp)
    else:
        tmp = [row.MachineName, row.Model, row.Denomination]
        slot_machines_libra.append(tmp)

In [315]:
#creazione della tabella con le decision variables
c_names = ["DecisionVar","MachineName","Model","Denomination"]
dec_var_libra = np.insert(slot_machines_libra,0,decision_variables_libra, axis = 1)
dec_var_df_libra = pd.DataFrame(dec_var_libra, columns = c_names)
dec_var_df_libra.head()

,DecisionVar,MachineName,Model,Denomination
0,x_1,Achaian,Predate,0.01
1,x_2,Achromatophil,Filum,2.0
2,x_3,Africander,Filum,0.01
3,x_4,Aleppo,Aeolian PRO V2,2.0
4,x_5,Amelanchier,Anosmia,0.02


### Matrice dei coefficienti di profitto

Per poter utilizzare i dati dei profitti medi giornalieri si crea una matrice formata da N righe quanto il totale delle slot machine presenti e da M colonne quante le section del casinò

In [317]:
# La seguente funzione prendere in ingresso MachineName, Denomination, Model e Section di una slot machine e ne restituisce
# il valore della profitability se esiste, 0 altrimenti
def prof_value_libra(name, den, model, section):
    riga = final_prof_libra[(final_prof_libra.MachineName == name) & (final_prof_libra.Model == model) & (final_prof_libra.Denomination == den)]
    if section in riga.Section.values:
        value = riga[riga.Section == section].Daily_profitability.values
        return float(value[0])
    else:
        return 1

In [318]:
#Creo un matrice a cui andrò ad aggiungere riga per riga i valori dei profitti
prof_matrix_libra = np.array((0,0,0,0))
for rownum,row in Libra_wh.iterrows():
    name = row.MachineName
    den = row.Denomination
    model = row.Model
    #Se ho più di 1 slot per configurazione creo tante righe quante le slot
    if row.NoMachines != 1:
        p_bound = prof_value_libra(name, den, model, 'Boundary')
        p_entrance = prof_value_libra(name, den, model, 'Entrance')
        p_interior = prof_value_libra(name, den, model, 'Interior')
        p_rest = prof_value_libra(name, den, model, 'RestaurantPlaza')
        for i in range(row.NoMachines):
            tmp = np.array([p_bound,p_entrance,p_interior,p_rest])
            prof_matrix_libra = np.vstack((prof_matrix_libra, tmp))
    else:
        p_bound = prof_value_libra(name, den, model, 'Boundary')
        p_entrance = prof_value_libra(name, den, model, 'Entrance')
        p_interior = prof_value_libra(name, den, model, 'Interior')
        p_rest = prof_value_libra(name, den, model, 'RestaurantPlaza')
        tmp = np.array([p_bound,p_entrance,p_interior,p_rest])
        prof_matrix_libra = np.vstack((prof_matrix_libra, tmp))
        
prof_matrix_libra = np.delete(prof_matrix_libra, 0, 0)

In [319]:
#Trasformo la matrice in un dataset in modo da poterci accedere con 
#i valori delle section e decision variables
profit_coeff_libra = pd.DataFrame(prof_matrix_libra, columns = section, index = decision_variables_libra)

## Creazione modello e inserimento funzione obiettivo

$
   X_{j,i} = 
    \begin{cases}
        1 & \quad \text{se la slot} ~ i ~\text{è presente nella sezione}~j\\
        0 & \quad \text{altrimenti}
    \end{cases}
\
$

$
    P_{j,i} = profitability[j,i] ~\text{dove profitability è la matrice dei profitti creata precedentemente}
$

In [320]:
prob_libra = pulp.LpProblem('Floor Mix Optimization at Libra Casino', pulp.LpMaximize)

#La funzione LpVariable.dicts crea un dizionario con tutte le possibili
#combinazioni tra i e j
is_machine_var_libra = pulp.LpVariable.dicts("is there",
                                      ((j, i) for j in section
                                              for i in decision_variables_libra),
                                      0, 1, LpBinary)

Funzione obiettivo:
   
$ \displaystyle \sum_{j \in section} \sum_{i \in decision_variables} X_{j,i}P_{j,i}$

In [321]:
prob_libra += (
    pulp.lpSum(
        [profit_coeff_libra[j][i] * is_machine_var_libra[(j,i)]]
        for j in section for i in decision_variables_libra
    )
)

## Vincoli del modello

Vengono aggiunti al modello i vincoli di capienza delle singole stanze e un vincolo per ogni variabile decisionale poichè rappresentando la singola slot presente nel Casinò può essere presente solamente in una delle 4 sezioni.

In [322]:
prob_libra += pulp.lpSum([is_machine_var_libra['Boundary', i] for i in decision_variables_libra]) == Nslot_libra_boundary #Boundary
prob_libra += pulp.lpSum([is_machine_var_libra['Entrance', i] for i in decision_variables_libra]) == Nslot_libra_entrance #Entrance
prob_libra += pulp.lpSum([is_machine_var_libra['Interior', i] for i in decision_variables_libra]) == Nslot_libra_interior #Interior
prob_libra += pulp.lpSum([is_machine_var_libra['RestaurantPlaza', i] for i in decision_variables_libra]) == Nslot_libra_restaurant #RestaurantPlaza
for i in decision_variables_libra:
    prob_libra += is_machine_var_libra['Boundary',i] + is_machine_var_libra['Entrance',i] + is_machine_var_libra['Interior',i] + is_machine_var_libra['RestaurantPlaza',i] == 1

## Calcolo Modello e soluzione ottimale

In [323]:
prob_libra.solve()

1

Controllo che la capienza delle singole sezioni sia corretta dopo la risoluzione del modello

In [324]:
tot = 0
for var in is_machine_var_libra:
    if var[0] == 'Boundary':
        var_value = is_machine_var_libra[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in Boundary: {0}".format(tot)

Numero di Slot in Boundary: 53.0


In [325]:
tot = 0
for var in is_machine_var_libra:
    if var[0] == 'Entrance':
        var_value = is_machine_var_libra[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in Entrance: {0}".format(tot)

Numero di Slot in Entrance: 56.0


In [326]:
tot = 0
for var in is_machine_var_libra:
    if var[0] == 'Interior':
        var_value = is_machine_var_libra[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in Interior: {0}".format(tot)

Numero di Slot in Interior: 39.0


In [327]:
tot = 0
for var in is_machine_var_libra:
    if var[0] == 'RestaurantPlaza':
        var_value = is_machine_var_libra[var].varValue
        tot += var_value
        #print "Sezione {0}, Slot {1}, VALORE: {2}".format(var[0],var[1], var_value)
        #print "Slot_name {0}, {1}".format(var[1], var_value)

print "Numero di Slot in RestaurantPlaza: {0}".format(tot)

Numero di Slot in RestaurantPlaza: 82.0


In [328]:
opt_result_libra = prob_libra.solve()

assert opt_result_libra == pulp.LpStatusOptimal
print("Status: ", LpStatus[prob_libra.status])
print("Optimal solution: ", round(prob_libra.objective.value(),2))
# print("Individual variables: ")
# for v in prob_libra.variables():
#     print(v.name, " = ", v.varValue)

('Status: ', 'Optimal')
('Optimal solution: ', 512223405.79)


### Confronto risultati con Settembre 2011

Confronto tra la disposizione e la profitability del casino del 2011-09-01 con quella ottimale calcolata


In [329]:
#table_col_names = ['MachineName','B_11', 'E_11', 'I_11', 'R_11','B_12', 'E_12', 'I_12', 'R_12']
df_libra_sept = df_libra[df_libra.Month == '2011-09-01']

old_disposizione_libra = np.array((0,0,0,0))

for rownum,row in dec_var_df_libra.iterrows():
    name = row.MachineName
    model = row.Model
    den = row.Denomination
    old_disp = df_libra_sept[(df_libra_sept.MachineName == name) & (df_libra_sept.Model == model) & (df_libra_sept.Denomination == float(den))].Section.values[0]
    if old_disp == 'Boundary':
        old_tmp = np.array((1,0,0,0))
    elif old_disp == 'Entrance':
        old_tmp = np.array((0,1,0,0))
    elif old_disp == 'Interior':
        old_tmp = np.array((0,0,1,0))
    else:
        old_tmp = np.array((0,0,0,1))
    old_disposizione_libra = np.vstack((old_disposizione_libra, old_tmp))
old_disposizione_libra = np.delete(old_disposizione_libra, 0, 0)
name_df_libra = dec_var_df_libra.MachineName
old_disposizione_df_libra = pd.DataFrame(old_disposizione_libra)

old_disposizione_df_libra = pd.concat([old_disposizione_df_libra, name_df_libra], axis = 1)
old_disposizione_df_libra['index'] = decision_variables_libra
old_disposizione_df_libra = old_disposizione_df_libra.set_index('index')
cols = old_disposizione_df_libra.columns.tolist()
cols = cols[-1:] + cols[:-1]
old_disposizione_df_libra = old_disposizione_df_libra[cols]
old_disposizione_df_libra[4] = 0
old_disposizione_df_libra[5] = 0
old_disposizione_df_libra[6] = 0
old_disposizione_df_libra[7] = 0

old_disposizione_df_libra.columns = table_col_names
old_disposizione_df_libra['B_11'] = old_disposizione_df_libra.B_11.astype(int)
old_disposizione_df_libra['E_11'] = old_disposizione_df_libra.E_11.astype(int)
old_disposizione_df_libra['I_11'] = old_disposizione_df_libra.I_11.astype(int)
old_disposizione_df_libra['R_11'] = old_disposizione_df_libra.R_11.astype(int)
old_disposizione_df_libra['B_12'] = old_disposizione_df_libra.B_12.astype(int)
old_disposizione_df_libra['E_12'] = old_disposizione_df_libra.E_12.astype(int)
old_disposizione_df_libra['I_12'] = old_disposizione_df_libra.I_12.astype(int)
old_disposizione_df_libra['R_12'] = old_disposizione_df_libra.R_12.astype(int)

for v in prob_libra.variables():
    indice = str(v)[23:-2]
    if indice.startswith('aza'):
        indice = str(v)[30:-2]
    if str(v)[11:19] == 'Boundary':
        sect = 'B_12'
    elif str(v)[11:19] == 'Entrance':
        sect = 'E_12'
    elif str(v)[11:19] == 'Interior':
        sect = 'I_12'
    else:
        sect = 'R_12'
    old_disposizione_df_libra.at[indice, sect] = int(v.varValue)

Rappresentazione sotto forma di DataFrame della disposizione iniziale e ottimale delle slot machines

In [330]:
colored_table_libra = old_disposizione_df_libra.loc[:,'B_11':'R_12'].style.applymap(lambda x: 'background-color: green' if x == 1 else 'background-color: lightgray')
colored_table_libra

Le seguenti righe di codice servono per aggiungere al DataFrame iniziale df_aries le righe corrispondenti ai dati della disposizione ottimale ed esportare il nuovo dataset in formato csv

In [331]:
dec_var_df_libra_ = dec_var_df_libra[:]
dec_var_df_libra_ = dec_var_df_libra_.set_index('DecisionVar')
dec_var_df_libra_['Section'] = ''
    
old_disposizione_df_libra_ = old_disposizione_df_libra[:]
old_disposizione_df_libra_['B_12'] = old_disposizione_df_libra_['B_12'].apply(change_b)
old_disposizione_df_libra_['E_12'] = old_disposizione_df_libra_['E_12'].apply(change_e)
old_disposizione_df_libra_['I_12'] = old_disposizione_df_libra_['I_12'].apply(change_i)
old_disposizione_df_libra_['R_12'] = old_disposizione_df_libra_['R_12'].apply(change_r)

for rownum,row in dec_var_df_libra_.iterrows():
    row['Section'] = old_disposizione_df_libra_.loc[rownum,'B_12':'R_12'].values[ old_disposizione_df_libra_.loc[rownum,'B_12':'R_12'].values != np.array(None)][0]

dec_var_df_libra_['Daily_profitability'] = 0.0
for rownum,row in dec_var_df_libra_.iterrows():
    name = row.MachineName
    model = row.Model
    deno = row.Denomination
    sec = row.Section
    try:
        value = final_prof_libra[(final_prof_libra.MachineName == name) & (final_prof_libra.Model == model) & (final_prof_libra.Denomination == float(deno)) & (final_prof_libra.Section == sec)].Daily_profitability.values[0]  
    except IndexError:
        value = final_prof_libra[(final_prof_libra.MachineName == name) & (final_prof_libra.Model == model) & (final_prof_libra.Denomination == float(deno))].Daily_profitability.values[0]
    dec_var_df_libra_.at[rownum,'Daily_profitability'] = value

dec_var_df_libra_['NoMachines'] = 1

In [333]:
dec_var_df_libra_final = dec_var_df_libra_.groupby(by = ['MachineName','Model','Denomination','Section'], as_index = False)['Daily_profitability','NoMachines'].sum()
dec_var_df_libra_final['Month'] = '2012-09-01'


dec_var_df_libra_final = dec_var_df_libra_final[['MachineName','Section','Denomination','Model','Month','NoMachines','Daily_profitability']]
dec_var_df_libra_final['Daily_profitability'] = dec_var_df_libra_final.Daily_profitability / dec_var_df_libra_final.NoMachines
df_libra[['MachineName','Section','Denomination','Model','Month','NoMachines','Daily_profitability']]
df_libracopia = df_libra[['MachineName','Section','Denomination','Model','Month','NoMachines','Daily_profitability']]

TUTTO_LIBRA = pd.concat([df_libracopia,dec_var_df_libra_final])
TUTTO_LIBRA['Profit'] = TUTTO_LIBRA.NoMachines*TUTTO_LIBRA.Daily_profitability

In [334]:
file_name_libra = folder_path + '\\Libra_casino.csv'     
TUTTO_LIBRA.to_csv(file_name_libra, encoding = 'utf-8', index = False)